In [1]:
# Application of FL task
from MLModel import *
from FLModel import *
from utils import *

from torchvision import datasets, transforms
import torch
import numpy as np
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [2]:
def load_cnn_mnist(num_users):
    data_train = datasets.MNIST(root="~/data/", train=True, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ]))

    data_test = datasets.MNIST(root="~/data/", train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ]))

    # split MNIST (training set) into non-iid data sets
    non_iid = []
    user_dict = mnist_noniid(data_train, num_users)
    for i in range(num_users):
        idx = user_dict[i]
        d = data_train.data[idx].float().unsqueeze(1)
        targets = data_train.targets[idx].float()
        non_iid.append((d, targets))
    non_iid.append((data_test.data.float().unsqueeze(1), data_test.targets.float()))
    return non_iid

In [3]:
"""
1. load_data
2. generate clients (step 3)
3. generate aggregator
4. training
"""
client_num = 10
d = load_cnn_mnist(client_num)

In [4]:
lr = 0.1
fl_param = {
    'output_size': 10,
    'client_num': client_num,
    'model': MnistCNN,
    'data': d,
    'lr': lr,
    'E': 1,
    'C': 1,
    'eps': 4.0,
    'delta': 1e-5,
    'q': 0.05,
    'clip': 32,
    'tot_T': 150,
    'batch_size': 128,
    'device': device
}
import warnings
warnings.filterwarnings("ignore")
fl_entity = FLServer(fl_param).to(device)

DP-SGD with sampling rate = 5% and noise_multiplier = 3.2772959917700293 iterated over 3000 steps satisfies differential privacy with eps = 4 and delta = 1e-05.
DP-SGD with sampling rate = 5% and noise_multiplier = 3.2772959917700293 iterated over 3000 steps satisfies differential privacy with eps = 4 and delta = 1e-05.
DP-SGD with sampling rate = 5% and noise_multiplier = 3.2772959917700293 iterated over 3000 steps satisfies differential privacy with eps = 4 and delta = 1e-05.
DP-SGD with sampling rate = 5% and noise_multiplier = 3.2772959917700293 iterated over 3000 steps satisfies differential privacy with eps = 4 and delta = 1e-05.
DP-SGD with sampling rate = 5% and noise_multiplier = 3.2772959917700293 iterated over 3000 steps satisfies differential privacy with eps = 4 and delta = 1e-05.
DP-SGD with sampling rate = 5% and noise_multiplier = 3.2772959917700293 iterated over 3000 steps satisfies differential privacy with eps = 4 and delta = 1e-05.
DP-SGD with sampling rate = 5% and

In [5]:
print("mnist")
acc = []
for e in range(150):
    fl_entity.set_lr(lr/np.sqrt(e+1))
    acc += [fl_entity.global_update()]
    print("global epochs = {:d}, acc = {:.4f}".format(e+1, acc[-1]))

mnist
global epochs = 1, acc = 0.0983
global epochs = 2, acc = 0.2993
global epochs = 3, acc = 0.3321
global epochs = 4, acc = 0.3000
global epochs = 5, acc = 0.3962
global epochs = 6, acc = 0.3166
global epochs = 7, acc = 0.4749
global epochs = 8, acc = 0.4213
global epochs = 9, acc = 0.5489
global epochs = 10, acc = 0.6332
global epochs = 11, acc = 0.7343
global epochs = 12, acc = 0.7141
global epochs = 13, acc = 0.7374
global epochs = 14, acc = 0.7830
global epochs = 15, acc = 0.7880
global epochs = 16, acc = 0.7864
global epochs = 17, acc = 0.7877
global epochs = 18, acc = 0.8195
global epochs = 19, acc = 0.8176
global epochs = 20, acc = 0.8271
global epochs = 21, acc = 0.8234
global epochs = 22, acc = 0.8366
global epochs = 23, acc = 0.8337
global epochs = 24, acc = 0.8342
global epochs = 25, acc = 0.8367
global epochs = 26, acc = 0.8409
global epochs = 27, acc = 0.8453
global epochs = 28, acc = 0.8413
global epochs = 29, acc = 0.8408
global epochs = 30, acc = 0.8450
global epochs